### Huanyu Li

# Part-of-Speech Tagging with Recurrent Neural Networks

Your task in this assignment is to implement a simple part-of-speech tagger based on recurrent neural networks.

## Part-of-speech tagging

Part-of-speech (POS) tagging is the task of labelling words (tokens) with [parts of speech](https://en.wikipedia.org/wiki/Part_of_speech). To give an example, consider the  sentence *Parker hates parsnips*. In this sentence, the word *Parker* should be labelled as a proper noun (a noun that is the name of a person), *hates* should be labelled as a verb, and *parsnips* should be labelled as a (common) noun. Part-of-speech tagging is an essential ingredient of many state-of-the-art natural language understanding systems.

Part-of-speech tagging can be cast as a supervised machine learning problem where the gold-standard data consists of sentences whose words have been manually annotated with parts of speech. For the present assignment you will be using a corpus built over the source material of the [English Web Treebank](https://catalog.ldc.upenn.edu/ldc2012t13), consisting of approximately 16,000&nbsp;sentences with 254,000&nbsp;tokens. The corpus has been released by the [Universal Dependencies Project](http://universaldependencies.org).

To make it easier to compare systems, the gold-standard data has been split into three parts: training, development (validation), and test. The following code uses three functions from the helper module `utils` (provided with this assignment) to load the data:

In [1]:
import utils

training_data = list(utils.read_training_data())
print('Number of sentences in the training data: {}'.format(len(training_data)))

development_data = list(utils.read_development_data())
print('Number of sentences in the development data: {}'.format(len(development_data)))

test_data = list(utils.read_test_data())
print('Number of sentences in the test data: {}'.format(len(test_data)))

Number of sentences in the training data: 12543
Number of sentences in the development data: 2002
Number of sentences in the test data: 2077


From a Python perspective, each of the data sets is a list of what we shall refer to as *tagged sentences*. A tagged sentence, in turn, is a list of pairs $(w,t)$, where $w$ is a word token and $t$ is the word&rsquo;s POS tag. Here is an example from the training data to show you how this looks like:

In [2]:
training_data[42]

[(b'There', b'PRON'),
 (b'has', b'AUX'),
 (b'been', b'VERB'),
 (b'talk', b'NOUN'),
 (b'that', b'SCONJ'),
 (b'the', b'DET'),
 (b'night', b'NOUN'),
 (b'curfew', b'NOUN'),
 (b'might', b'AUX'),
 (b'be', b'AUX'),
 (b'implemented', b'VERB'),
 (b'again', b'ADV'),
 (b'.', b'PUNCT')]

You will see part-of-speech tags such as `VERB` for verb, `NOUN` for noun, and `ADV` for adverb. If you are interested in learning more about the tag set used in the gold-standard data, you can have a look at the documentation of the [Universal POS tags](http://universaldependencies.org/u/pos/all.html). However, you do not need to understand the meaning of the POS tags to solve this assignment; you can simply treat them as labels drawn from a finite set of alternatives.

## Problem specification

Your task in this assignment is to build a part-of-speech tagger based on a recurrent neural network architecture, to train this tagger on the provided training data, and to evaluate its performance on the test data. To tune the hyperparameters of the network, you can use the provided development (validation) data.

### Network architecture

The proposed network architecture for your tagger is a sequential model with three layers, illustrated below: an embedding, a bidirectional LSTM, and a softmax layer. The embedding turns word indexes (integers representing words) into fixed-size dense vectors which are then fed into the bidirectional LSTM. The output of the LSTM at each position of the sentence is passed to a softmax layer which predicts the POS tag for the word at that position.

![System architecture](architecture.png)

To implement the network architecture, you will use [Keras](https://keras.io), a high-level neural network library for Python. Keras comes with an extensive online documentation, and reading the relevant parts of this documentation will be essential when working on this assignment. We suggest to start with the tutorial [Getting started with the Keras Sequential model](https://keras.io/getting-started/sequential-model-guide/). We also suggest to have a look at concrete examples, such as  [imdb_lstm.py](https://github.com/fchollet/keras/blob/master/examples/imdb_lstm.py).

### Pre-processing the data

Before you can start to implement the network architecture as such, you will have to bring the tagged sentences from the gold-standard data into a form that can be used with the network. At its core, this involves encoding each word and each tag as an index into a finite set (a non-negative integer), which can be done for example via a Python dictionary. Here is some code to illustrate the basic idea:

In [3]:
# Construct a simple index for words
w2i = dict()
for tagged_sentence in training_data:
    for word, tag in tagged_sentence:
        if word not in w2i:
            w2i[word] = len(w2i)    # assign next available index

print('Number of unique words in the training data: {}'.format(len(w2i)))
print('Index of the word "hates": {}'.format(w2i[b'hates']))

Number of unique words in the training data: 19672
Index of the word "hates": 4579


Once you have indexes for the words and the tags, you can construct the input and the gold-standard output tensor required to train the network.

**Constructing the input tensor.** The input tensor should be of shape $(N, n)$ where $N$ is the total number of sentences in the training data and $n$ is the length of the longest sentence. Note that Keras requires all sequences in an input tensor to have the same length, which means that you will have to pad all sequences to that length. You can use the helper function `pad_sequences` for this, which by default will front-pad sequences with the value&nbsp;0. It is essential then that you do not use this special padding value as the index of actual words.

**Constructing the gold-standard output tensor.** The gold-standard output tensor should be of shape $(N, n, T)$ where $T$ is the number of unique tags in the training data, plus one to cater for the special padding value. The additional dimension corresponds to the fact that the softmax layer of the network will output one $T$-dimensional vector for each position of an input sentence. To construct the gold-standard version of this vector, you can use the helper function `to_categorical`, which will produce a &lsquo;one-hot vector&rsquo; for a given tag index.

### Constructing the network

To implement the network architecture, you need to find and instantiate the relevant building blocks from the Keras library. Note that Keras layers support a large number of optional parameters; use the default values unless you have a good reason not to. Two mandatory parameters that you will have to specify are the dimensionality of the embedding and the dimensionality of the output of the LSTM layer. The following values are reasonable starting points:

* dimensionality of the embedding: 100
* dimensionality of the output of the bidirectional LSTM layer: 100

You will also have to choose an appropriate loss function. For training we recommend the Adam optimiser.

### Evaluation

The last problem that you will have to solve is to write code to evaluate the trained tagger. The most widely-used evaluation measure for part-of-speech tagging is per-word accuracy, which is the percentage of words to which the tagger assigns the correct tag (according to the gold standard). Implementing this metric should be straightforward. However, make sure that you remove (or ignore) the special padding value when you compute the tagging accuracy.

**The performance goal for this assignment is to build a tagger that achieves a development set accuracy of at least 90%.**

**Unknown words.** One problem that you will encounter during evaluation is that the development data contains words that you did not see (and did not add to your index) during training. The simplest solution to this problem is to reserve a special index for &lsquo;the unknown word&rsquo; which the network can use whenever it encounters an unknown word. When you go for this strategy, the size of your index will be equal to the number of unique words in the training data plus&nbsp;2 &ndash; one extra for the unknown word, and one for the padding symbol.

## Skeleton code

The following skeleton code provides you with a starting point for your implementation:

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, Bidirectional, Embedding, TimeDistributed
from keras.preprocessing import sequence
from keras.utils import to_categorical, plot_model
import keras.backend as K

#import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

class Tagger(object):

    def __init__(self):
        self.model = None
        self.training_w2i = dict()
        self.training_t2i = dict()
        self.max_len = 0

    def train(self, training_data):
        # Pre-process the training data
        # Construct the network, add layers, compile, and fit
        self.preprocess(training_data)
        self.max_len = len(max(training_data, key = len))
        input_sequences = self.construct_input_tensors(training_data)
        categorical_output = self.construct_output_tensors(training_data)
        self.model = self.construct_net()
        self.model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy', self.mean_pred])
        self.model.fit(input_sequences, categorical_output, batch_size = 32, epochs = 6)

    def preprocess(self, training_data):
        for tagged_sentence in training_data:
            for word, tag in tagged_sentence:
                if word not in self.training_w2i:
                    self.training_w2i[word] = len(self.training_w2i) + 1    # assign next available index
                if tag not in self.training_t2i:
                    self.training_t2i[tag] = len(self.training_t2i) + 1
        self.training_w2i['-UNKNOWN-'] = len(self.training_w2i) + 1
   
    def construct_input_tensors(self, data):
        input_tensor = list()
        for tagged_sentence in data:
            word_tensor = list()
            for word, tag in tagged_sentence:
                if word not in self.training_w2i.keys():
                    word_tensor.append(self.training_w2i['-UNKNOWN-'])
                else:
                    word_tensor.append(self.training_w2i[word])
            input_tensor.append(word_tensor)
        input_sequences = sequence.pad_sequences(input_tensor, maxlen = self.max_len)
        return input_sequences
    
    def construct_output_tensors(self, data):
        output_tensor = list()
        for tagged_sentence in data:
            tag_tensor = list()
            for word, tag in tagged_sentence:
                tag_tensor.append(self.training_t2i[tag])
            output_tensor.append(tag_tensor)
        output_sequences = sequence.pad_sequences(output_tensor, maxlen = self.max_len)
        return to_categorical(output_sequences)
    
    #https://keras.io/metrics/#custom-metrics
    def mean_pred(self, y_true, y_pred):
        #true_tag_tensor: [1,2,3,0,0,0]
        #pred_tag_tensor: [4,2,3,0,0,0]
        #last 3 are paddings
        true_tag_tensor = K.argmax(y_true, axis = -1)
        pred_tag_tensor = K.argmax(y_pred, axis = -1)
        #sequence_tag = [1,1,1,0,0,0]
        sequence_tag = K.cast(K.not_equal(pred_tag_tensor, 0), 'int64')
        #matches = [0,1,1,1,1,1]
        matches = K.cast(K.equal(true_tag_tensor, pred_tag_tensor), 'int64')
        #cleaned_matches = [0,1,1,0,0,0]
        cleaned_matches = matches * sequence_tag
        acc = K.cast(K.sum(cleaned_matches), 'float64') / (K.cast(K.sum(sequence_tag), 'float64') + K.epsilon())
        return acc
    
    def construct_net(self):
        #+1 because of padding, did not store padding in training_w2i and training_t2i
        embedding_input_dim = len(self.training_w2i) + 1
        embedding_output_dim = 100
        dense_input_dim = len(self.training_t2i) + 1
        model = Sequential()
        model.add(Embedding(embedding_input_dim, embedding_output_dim))
        model.add(Bidirectional(LSTM(100, return_sequences = True)))
        model.add(TimeDistributed(Dense(dense_input_dim, activation = 'softmax')))
        #plot_model(model, to_file='model.png')
        model.summary()
        return model
    
    def predict(self, data):
        data_input = self.construct_input_tensors(data)
        prediction = self.model.predict(data_input)
        return prediction
    
    def evaluate(self, gold_data):
        gold_input = self.construct_input_tensors(gold_data)
        gold_output = self.construct_output_tensors(gold_data)
        prediction = self.model.predict(gold_input)
        return K.eval(self.mean_pred(gold_output, prediction))

Using TensorFlow backend.


And here is how the tagger is supposed to be used:

In [5]:
tagger = Tagger()
tagger.train(training_data)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 100)         1967400   
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 200)         160800    
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 18)          3618      
Total params: 2,131,818
Trainable params: 2,131,818
Non-trainable params: 0
_________________________________________________________________


/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/6
12543/12543 [==============================] - 145s 12ms/step - loss: 0.3062 - accuracy: 0.9217 - mean_pred: 0.2847
Epoch 2/6
12543/12543 [==============================] - 123s 10ms/step - loss: 0.0596 - accuracy: 0.9837 - mean_pred: 0.8473
Epoch 3/6
12543/12543 [==============================] - 114s 9ms/step - loss: 0.0214 - accuracy: 0.9943 - mean_pred: 0.9442
Epoch 4/6
12543/12543 [==============================] - 116s 9ms/step - loss: 0.0137 - accuracy: 0.9961 - mean_pred: 0.9617
Epoch 5/6
12543/12543 [==============================] - 117s 9ms/step - loss: 0.0104 - accuracy: 0.9969 - mean_pred: 0.9701
Epoch 6/6
12543/12543 [==============================] - 127s 10ms/step - loss: 0.0082 - accuracy: 0.9976 - mean_pred: 0.9767


In [6]:
acc = tagger.evaluate(development_data)
print('Accuracy on development data: {:.1%}'.format(acc))

Accuracy on development data: 90.3%


## Submission

Submit this assignment by emailing this notebook to Marco. Your notebook should include all your code, and should be runnable by Marco without further modification. It should also include a short text with your reflections on this assignment. What did you find particularly surprising or hard? What have you learned from this assignment? You can paste your text into the box below. Good luck!

### Reflections

In the beginning, the accuracy of the development data was extremely high. I thought it was not correct, especially after I checked the predicted result for some samples, then I saw the result was not correct. Then I realized it was because of the padding zeros in the tensors. It took me some time to write the customed metric function.  As I started to do the lab before the lab session and didn't find there is a way to set MASK_ZERO as True to avoid writing my own customed metric for the evaluation. I have to write the customed metric mean_pred to ignore the padding zeros first, then calculate the accuracy. In the end, the network can achieve accuracy in 90.5% after trained with 32 batch size and six epochs. 

What I have learned from this assignment is, how to use some basic functions from Keras to build a neural network for this particular question. What is a little bit not easy to do is to figure out meanings for some default parameters of the classes in Keras. 
Considering the POS lab I did in TDDE09 (the data is the same, I think), the best performance of the multi-class perceptron can reach 87%. After some feature engineering work, the performance reaches 90%, which is close to the result of the neural network in this lab. It can be explained the former method takes limited features of the sequence even if it uses feature engineering, while the latter RNN based neural network considers the whole sequence. 
